# Linear Regression

### Loading Libraries

In [ ]:
# Numerical Computing
import numpy as np


# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
from matplotlib .pyplot import subplots

# StatsModel
import statsmodels .api as sm

# ISLP - Custom Libraries
from ISLP import load_data
from ISLP.models import ( ModelSpec as MS,
summarize,
poly)

In [ ]:
from statsmodels .stats. outliers_influence \
import variance_inflation_factor as VIF
from statsmodels .stats.anova import anova_lm

#### Inspecting Objects & Namespaces

In [ ]:
dir ()

In [ ]:
A = np.array ([3 ,5 ,11])

dir(A)

In [ ]:
A.sum ()

### Simple Linear Regression

In [ ]:
Boston = load_data("Boston")

Boston.columns

In [ ]:
X = pd. DataFrame ({'intercept': np.ones(Boston.shape [0]),
                    'lstat': Boston['lstat']})

X[:4]

In [ ]:
y = Boston['medv']

model = sm.OLS(y, X)
results = model.fit ()

In [ ]:
summarize(results)

#### Using Transformations: Fit & Transform

In [ ]:
design = MS(['lstat'])
design = design.fit(Boston)

X = design. transform (Boston)
X[:4]

In [ ]:
design = MS(['lstat'])

X = design. fit_transform (Boston)
X[:4]

In [ ]:
# results.summary()

print(results.summary())

In [ ]:
results.params

In [ ]:
new_df = pd.DataFrame({'lstat':[5, 10, 15]})

newX = design.transform(new_df)
newX

In [ ]:
new_predictions = results. get_prediction(newX);

new_predictions . predicted_mean

In [ ]:
new_predictions . conf_int (alpha =0.05)

In [ ]:
new_predictions.conf_int (obs=True , alpha =0.05)

#### Defining Functions

In [ ]:
def abline(ax , b, m):
    "Add a line with slope m and intercept b to ax"
    xlim = ax. get_xlim ()
    ylim = [m * xlim [0] + b, m * xlim [1] + b]
    ax.plot(xlim , ylim)

In [ ]:
def abline(ax , b, m, *args , ** kwargs):
    "Add a line with slope m and intercept b to ax"
    xlim = ax. get_xlim ()
    ylim = [m * xlim [0] + b, m * xlim [1] + b]
    ax.plot(xlim , ylim , *args , ** kwargs)

In [ ]:
ax = Boston.plot.scatter('lstat', 'medv')

abline(ax,
       results.params [0],
       results.params [1],
       'r--',
       linewidth =3)

plt.grid(True)
plt.show()

In [ ]:
ax = subplots (figsize =(8 ,8))[1]

ax.scatter(results.fittedvalues , results.resid, color='gold')
ax. set_xlabel ('Fitted value')
ax. set_ylabel ('Residual')
ax.axhline (0, c='k', ls='--');
plt.grid(True)
plt.show()

In [ ]:
infl = results. get_influence ()

ax = subplots (figsize =(8 ,8))[1]
ax.scatter(np.arange(X.shape [0]) , infl. hat_matrix_diag, color='tomato')
ax. set_xlabel ('Index')
ax. set_ylabel ('Leverage')
np.argmax(infl. hat_matrix_diag)
plt.grid(True)
plt.show()

### Multiple Linear Regression

In [ ]:
X = MS(['lstat', 'age']). fit_transform (Boston)

model1 = sm.OLS(y, X)
results1 = model1.fit ()

In [ ]:
summarize(results1)

In [ ]:
terms = Boston.columns.drop('medv')
terms

In [ ]:
X = MS(terms). fit_transform (Boston)

model = sm.OLS(y, X)
results = model.fit ()

In [ ]:
summarize(results)

In [ ]:
minus_age = Boston.columns.drop (['medv', 'age'])

Xma = MS( minus_age ). fit_transform (Boston)
model1 = sm.OLS(y, Xma)

In [ ]:
summarize(model1.fit())

### Multivariate Goodness of Fit

#### List Comprehension

In [ ]:
vals = [VIF(X, i)
        for i in range (1, X.shape [1])]

vif = pd. DataFrame ({'vif':vals},
                     index=X.columns [1:])

vif

In [ ]:
vals = []

for i in range (1, X.values.shape [1]):
    vals.append(VIF(X.values , i))

### Interaction Terms

In [ ]:
X = MS(['lstat',
        'age',
        ('lstat', 'age')]). fit_transform (Boston)

model2 = sm.OLS(y, X)

In [ ]:
summarize(model2.fit ())

### Non-linear Transformations of the Predictors

In [ ]:
X = MS([ poly('lstat', degree =2), 'age']).fit_transform(Boston)

model3 = sm.OLS(y, X)
results3 = model3.fit()

In [ ]:
summarize(results3)

In [ ]:
anova_lm(results1 , results3)

In [ ]:
ax = subplots(figsize =(8 ,8))[1]

ax.scatter(results3 .fittedvalues, results3.resid, color='royalblue')
ax. set_xlabel ('Fitted value')
ax. set_ylabel ('Residual')
ax.axhline (0, c='k', ls='--')

plt.grid(True)
plt.show()

### Qualitative Predictors

In [ ]:
import statsmodels.api as sm
from ISLP.models import ModelSpec as MS
from ISLP.models.model_spec import Interaction

Carseats = load_data ('Carseats')

Carseats.columns

In [ ]:
# allvars = list(Carseats .columns.drop('Sales'))

# y = Carseats ['Sales']

# final = allvars + [('Income', 'Advertising'),
#                    ('Price ', 'Age ')]

# X = MS(final).fit_transform(Carseats)
# model = sm.OLS(y, X)

In [ ]:
Carseats.columns = Carseats.columns.str.strip()

Carseats["Income_x_Advertising"] = Carseats["Income"] * Carseats["Advertising"]
Carseats["Price_x_Age"] = Carseats["Price"] * Carseats["Age"]

y = Carseats["Sales"]
allvars = list(Carseats.columns.drop("Sales"))

spec = MS(allvars)
X = spec.fit_transform(Carseats)

if not hasattr(X, "columns") or "Intercept" not in X.columns:
    X = sm.add_constant(X)

# 6) OLS
model = sm.OLS(y, X).fit()
print(model.summary())